In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
from huggingface_hub import login
login(token = HF_TOKEN)

In [3]:
!pip install -q -U unsloth
from unsloth import FastLanguageModel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2025-06-25 15:22:49.268212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750864969.461246      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750864969.515351      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model_name = "ParamDev/mistral-7b-v0.3_alpaca"

# Load model with tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None,              # "auto" by default; set to torch.float16 or bfloat16 if needed
    load_in_4bit = True        # for quantized inference (efficient)
)

# Speed up with Flash Attention 2 (optional)
FastLanguageModel.for_inference(model)


==((====))==  Unsloth 2025.6.5: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2025.6.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096, padding_idx=770)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [5]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# outputs = model.generate(**inputs, max_new_tokens=100)

# response = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(response)


In [6]:
# prompt = "<s>[INST] {}\n{} [/INST] {} </s>"

# inputs = tokenizer(
# [
#     prompt.format(
#         "Continue the fibonnaci sequence.", # instruction
#         "1, 1, 2, 3, 5, 8", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
# generated_text =  tokenizer.batch_decode(outputs)

# # Extract only what's after "</s>"
# if "</s>" in generated_text[0]:
#     response = generated_text[0].split("</s>")[1].strip()
# else:
#     response = generated_text[0].strip()

# print("Extracted response:\n" + response)

In [7]:
# print("Extracted response:\n" ,generated_text)


In [8]:
def run_inference(instruction, input_text):
    prompt = "<s>[INST] {}\n{} [/INST] {} </s>"
    prompt = prompt.format(instruction, input_text, "")
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    decoded = tokenizer.batch_decode(outputs)[0]

    if "</s>" in decoded:
        return decoded.split("</s>")[1].strip()
    return decoded.strip()


In [9]:
# examples = [
#     ("Continue the Fibonacci sequence.", "1, 1, 2, 3, 5, 8"),
#     ("Solve the following math problem.", "If a train travels 60 km in 1.5 hours, what is its speed in km/h?"),
#     ("Write a Python function to check if a number is prime.", "Use efficient logic, not brute force."),
#     ("Rewrite the sentence in passive voice.", "The manager approved the budget."),
#     ("Translate the following sentence to French.", "The weather is very nice today."),
#     ("Explain what this Python code does.", "for i in range(5): print(i*i)"),
#     ("Describe the importance of the ozone layer.", ""),
#     ("Write a short story about a time-traveling cat.", ""),
# ]

# for i, (instruction, input_text) in enumerate(examples, 1):
#     print(f"\n🧠 Example {i}: {instruction}\n{input_text}")
#     result = run_inference(instruction, input_text)
#     print("Response:", result)


In [10]:
import torch

# 1. Create a small evaluation dataset with coding prompts
eval_dataset = [
    {
        "instruction": "Write a Python function to check if a number is a palindrome.",
        "input": "The function should take an integer and return True if it's a palindrome, False otherwise.",
        "reference": "def is_palindrome(n):\n    return str(n) == str(n)[::-1]"
    },
    {
        "instruction": "Complete the Python function to calculate the factorial of a number.",
        "input": "def factorial(n):\n    # Add your code here",
        "reference": "def factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)"
    },
    {
        "instruction": "Translate the following Python list comprehension to a regular for loop.",
        "input": "squares = [x*x for x in range(10)]",
        "reference": "squares = []\nfor x in range(10):\n    squares.append(x*x)"
    }
]

# 2. Helper function to generate responses from your model
def generate_responses(dataset, model, tokenizer):
    predictions = []
    alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""
    print("Generating model predictions...")
    for item in dataset:
        prompt = alpaca_prompt_template.format(item["instruction"], item["input"])
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True, pad_token_id=tokenizer.eos_token_id)
        full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        try:
            prediction = full_response.split("### Response:")[1].strip()
        except IndexError:
            prediction = ""
        
        predictions.append(prediction)
        print(f"Instruction: {item['instruction']}\n- Model Output: {prediction}\n---")
        
    return predictions

# 3. Generate the predictions and references (This creates the missing variables)
predictions = generate_responses(eval_dataset, model, tokenizer)
references = [item['reference'] for item in eval_dataset]

print("\n\n--- Predictions and References have been successfully generated! ---")
print("--- Now calculating scores... ---\n")

Generating model predictions...
Instruction: Write a Python function to check if a number is a palindrome.
- Model Output: ```python
def is_palindrome(num):
    # Convert the number to a string
    num_str = str(num)
    # Reverse the string
    reversed_num_str = num_str[::-1]
    # Compare the original string and the reversed string
    if num_str == reversed_num_str:
        return True
    else:
        return False
```

This function takes an integer as input and checks if it is a palindrome. It converts the integer to a string, reverses
---
Instruction: Complete the Python function to calculate the factorial of a number.
- Model Output: Here is the completed function that calculates the factorial of a number:

```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```

### Explanation:
The instruction asks to complete the Python function to calculate the factorial of a number. The input provides the function definition, but it is m

In [11]:
!pip install -q evaluate rouge_score sacrebleu bert_score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00


In [12]:
import evaluate

# --- BLEU Score ---
bleu_metric = evaluate.load("bleu")
# BLEU expects a list of lists for references
bleu_results = bleu_metric.compute(predictions=predictions, references=[[r] for r in references])
print("--- BLEU Score ---")
print(f"BLEU: {bleu_results['bleu']:.4f}")
print(f"Precisions: {bleu_results['precisions']}")


# --- ROUGE Score ---
rouge_metric = evaluate.load("rouge")
rouge_results = rouge_metric.compute(predictions=predictions, references=references)
print("\n--- ROUGE Score ---")
print(f"ROUGE-1: {rouge_results['rouge1']:.4f} (unigram overlap)")
print(f"ROUGE-2: {rouge_results['rouge2']:.4f} (bigram overlap)")
print(f"ROUGE-L: {rouge_results['rougeL']:.4f} (longest common subsequence)")

--- BLEU Score ---
BLEU: 0.1646
Precisions: [0.20064724919093851, 0.17647058823529413, 0.15181518151815182, 0.13666666666666666]



--- ROUGE Score ---
ROUGE-1: 0.2348 (unigram overlap)
ROUGE-2: 0.1799 (bigram overlap)
ROUGE-L: 0.2259 (longest common subsequence)


In [13]:
# Install the necessary library (it's good practice to keep it in the cell)
!pip install -q bert_score

import evaluate

# Correctly load the "bertscore" metric (no underscore)
bertscore_metric = evaluate.load("bertscore")
bert_results = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

# We get an average over all the samples
avg_precision = sum(bert_results['precision']) / len(bert_results['precision'])
avg_recall = sum(bert_results['recall']) / len(bert_results['recall'])
avg_f1 = sum(bert_results['f1']) / len(bert_results['f1'])

print("\n--- BERT-Score ---")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall:    {avg_recall:.4f}")
print(f"Average F1 Score:  {avg_f1:.4f}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- BERT-Score ---
Average Precision: 0.8368
Average Recall:    0.9510
Average F1 Score:  0.8901
